# **Import Libraries**

In [1]:
#Import Libraries
import numpy as np
import itertools
import queue
import pandas as pd
from IPython.display import display, Markdown
import random
import sys
import time
import math
import tqdm
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import csv

# **Import Dataset**

In [2]:
filename = "your_dataset.csv"

f = open(filename, 'r')
l = csv.reader(f)

# number of jobs
n = int(next(l)[0])

# number of machines
m = int(next(l)[0])

# Read the processing times from the CSV file
processing_time_job_on_machine = []
for _ in range(n):
    row = list(map(int, next(l)))
    processing_time_job_on_machine.append(row)

# Convert the processing times to a NumPy array for easier manipulation
processing_time_job_on_machine = np.array(processing_time_job_on_machine)

# Calculate the processing time of each job on all machines
Processing_times = []
for job_index in range(n):
    processing_time_per_machine = []
    for machine_index in range(m):
        processing_time = processing_time_job_on_machine[job_index, machine_index]
        processing_time_per_machine.append(processing_time)
    Processing_times.append(processing_time_per_machine)

f.close()

# **Initial Population**

In [3]:
# Initial population
def initialization(Ps):
    pop = []
    for i in range(Ps):
        p = list(np.random.permutation(n))
        while p in pop:
            p = list(np.random.permutation(n))
        pop.append(p)
    return pop

# **Calculate Objective Value**

In [4]:
def calculateObj(sol):
    qTime = queue.PriorityQueue()

    qMachines = []
    for i in range(m):
        qMachines.append(queue.Queue())

    for i in range(n):
        qMachines[0].put(sol[i])

    busyMachines = []
    for i in range(m):
        busyMachines.append(False)

    time = 0

    job = qMachines[0].get()
    qTime.put((time+Processing_times[job][0], 0, job))
    busyMachines[0] = True

    while True:
        time, mach, job = qTime.get()
        if job == sol[n-1] and mach == m-1:
            break
        busyMachines[mach] = False
        if not qMachines[mach].empty():
                j = qMachines[mach].get()
                qTime.put((time+Processing_times[j][mach], mach, j))
                busyMachines[mach] = True
        if mach < m-1:
            if busyMachines[mach+1] == False:
                qTime.put((time+Processing_times[job][mach+1], mach+1, job))
                busyMachines[mach+1] = True
            else:
                qMachines[mach+1].put(job)

    return time

# **Selection:**
**Tournoment Selection**



**Roulette wheel Selection**

In [5]:
######### Selection #########

def fitness(time):
    if time > 0:
        return 1/time



# Tournament selection
def tournament_selection(population, tournament_size):
    parents = []
    for _ in range(2):
        tournament_candidates = random.sample(population, tournament_size)
        tournament_winner = min(tournament_candidates, key=lambda x: calculateObj(x))
        parents.append(tournament_winner)
    return parents



# Roulette wheel selection
def roulette_wheel_selection(population):
    fitness_values = [fitness(calculateObj(ind)) for ind in population]
    total_fitness = sum(fitness_values)

    probabilities = [fit / total_fitness for fit in fitness_values]

    selected_indices = np.random.choice(len(population), size=2, p=probabilities)

    parents = [population[idx] for idx in selected_indices]

    return parents

# **Crossover:**

**Two point Crossover**

**Linear Order Crossover**

**partially mapped crossover**

**similar job order crossover**

In [6]:
######## Crossover #########

#Two point Crossover

def two_point_crossover(parents):
    pos = list(np.random.permutation(np.arange(n-1)+1)[:2])

    if pos[0] > pos[1]:
        pos[0], pos[1] = pos[1], pos[0]

    child1 = list(parents[0])
    child2 = list(parents[1])


    for i in range(pos[0], pos[1]):
        child1[i] = -1
        child2[i] = -1

    # Fill remaining elements by scanning the other parent
    p1, p2 = -1, -1
    for i in range(pos[0], pos[1]):
        while True:
            p1 += 1
            if parents[1][p1] not in child1:
                child1[i] = parents[1][p1]
                break

    for i in range(pos[0], pos[1]):
        while True:
            p2 += 1
            if parents[0][p2] not in child2:
              child2[i] = parents[0][p2]
              break


    return [child1, child2]




#Linear Order Crossover

def linearly_order_crossover(parents):
    pos = list(np.random.permutation(np.arange(n - 1) + 1)[:2])

    if pos[0] > pos[1]:
        pos[0], pos[1] = pos[1], pos[0]

    child1 = [-1] * n
    child2 = [-1] * n



    # Copy a segment from the first parent to the child1 and second parent to the child2
    child1[pos[0]:pos[1] + 1] = parents[0][pos[0]:pos[1] + 1]
    child2[pos[0]:pos[1] + 1] = parents[1][pos[0]:pos[1] + 1]

    # Fill in the remaining positions of child1 with the genes from the second parent and child2 with the genes from the first parent
    p1, p2 = -1, -1
    for i in range(n):
        if child1[i] == -1:
            while True:
                p1 += 1
                if parents[1][p1] not in child1:
                    child1[i] = parents[1][p1]
                    break

    for i in range(n):
        if child2[i] == -1:
            while True:
                p2 += 1
                if parents[0][p2] not in child2:
                    child2[i] = parents[0][p2]
                    break

    return [child1, child2]





# PMX (partially mapped crossover)

def partially_mapped_crossover(parents):
    pos = list(np.random.permutation(np.arange(n - 1) + 1)[:2])

    if pos[0] > pos[1]:
        pos[0], pos[1] = pos[1], pos[0]

    child1 = parents[0].copy()
    child2 = parents[1].copy()

    mapping_1 = parents[0][pos[0]:pos[1] + 1]
    mapping_2 = parents[1][pos[0]:pos[1] + 1]


    func_1 = list(zip(mapping_1, mapping_2))


    for i in range(len(child1)):
        for element_1, element_2 in func_1:
            if child1[i] == element_1:
                child1[i] = element_2
            elif child1[i] == element_2:
                child1[i] = element_1

    for i in range(len(child2)):
        for element_1, element_2 in func_1:
            if child2[i] == element_1:
                child2[i] = element_2
            elif child2[i] == element_2:
                child2[i] = element_1

    return [child1, child2]





# SJOX (similar job order crossover)

def similar_job_order_crossover(parents):

    pos = random.randrange(1, n-1)

    child1 = [-1]*len(parents[0])
    child2 = [-1]*len(parents[1])
    for idx, (a, b) in enumerate(list(zip(parents[0], parents[1]))):
      if a == b:
        child1[idx] = a
        child2[idx] = b


    child1[:pos] = parents[0][:pos]
    child2[:pos] = parents[1][:pos]

    in_parent2_not_in_child1 = [i for i in parents[1] if i not in child1]
    in_parent1_not_in_child2 = [i for i in parents[0] if i not in child2]


    j_1 = 0

    for i in range(len(child1)):
      if child1[i] == -1:
        child1[i] = in_parent2_not_in_child1[j_1]
        j_1 += 1


    j_2 = 0

    for i in range(len(child2)):
      if child2[i] == -1:
        child2[i] = in_parent1_not_in_child2[j_2]
        j_2 += 1

    return [child1, child2]

# **Mutation**

**Arbitrary three-jobs change**

**Arbitrary two-jobs change**

**Shift change**

In [7]:
####### Mutation #######

#Arbitrary three-jobs change

def three_job_change(sol):
    selected_jobs = random.sample(sol, 3)

    jobs_idx = [sol.index(job) for job in selected_jobs]

    random.shuffle(selected_jobs)

    z = 0
    for i in jobs_idx:
        sol[i] = selected_jobs[z]
        z += 1

    return sol



#Arbitrary two-jobs change

def two_job_change(sol):
    selected_jobs = random.sample(sol, 2)
    jobs_idx = [sol.index(job) for job in selected_jobs]

    random.shuffle(selected_jobs)

    z = 0
    for i in jobs_idx:
        sol[i] = selected_jobs[z]
        z += 1

    return sol



#Shift change

def shift_change(sol):
    pos = list(np.random.permutation(np.arange(n))[:2])

    if pos[0] > pos[1]:
        t = pos[0]
        pos[0] = pos[1]
        pos[1] = t

    remJob = sol[pos[1]]

    for i in range(pos[1], pos[0], -1):
        sol[i] = sol[i-1]

    sol[pos[0]] = remJob

    return sol

# **Elitist Strategy**

In [ ]:
############# Elitist Strategy ###############

def elitistUpdate(oldPop, newPop):
    C_max_old = []
    for j in range(len(oldPop)):
        C_max_old.append(calculateObj(oldPop[j]))

    oldPop_Cmax = list(zip(oldPop,C_max_old))

    sorted_oldPop_Cmax = sorted(oldPop_Cmax, key=lambda x: x[1])
#    oldPop = sorted_oldPop_Cmax[:][0]
    oldPop = [element[0] for element in sorted_oldPop_Cmax]
    C_max_new = []
    for a in range(len(newPop)):
        C_max_new.append(calculateObj(newPop[a]))

    newPop_Cmax = list(zip(newPop, C_max_new))

    sorted_newPop_Cmax = sorted(newPop_Cmax, key=lambda x: x[1])
    newPop = [element[0] for element in sorted_newPop_Cmax]
    bestSolsIndex = [0, 1]
    bestSols = [calculateObj(oldPop[0]), calculateObj(oldPop[1])]


    for i in range(2, len(oldPop)):
        tempObj = calculateObj(oldPop[i])
        for idx in bestSolsIndex:
            if tempObj < bestSols[idx]:
                bestSols[idx] = tempObj
                bestSolsIndex[idx] = i

    newPop[-1] = oldPop[bestSolsIndex[0]]
    newPop[-2] = oldPop[bestSolsIndex[1]]

    return newPop

In [ ]:
def elitistUpdate(population, new_population):
    best_idx, _ = findBestSolution(population)
    worst_idx1, _ = findWorstSolution(population)
    worst_idx2, _ = findWorstSolution(population)

    # Replace two worst solutions with the best solution and a new random solution
    population[worst_idx1] = population[best_idx].copy()
    population[worst_idx2] = new_population.copy()

    return population

def findWorstSolution(population):
    worstObj = calculateObj(population[0])
    worstInd = 0
    for i in range(1, len(population)):
        tObj = calculateObj(population[i])
        if tObj > worstObj:
            worstObj = tObj
            worstInd = i
    return worstInd, worstObj

# **Find Best Solution**

In [8]:
# Returns best solution's index number, best solution's objective value and average objective value of the given population.
def findBestSolution(pop):
    bestObj = calculateObj(pop[0])
    bestInd = 0
    for i in range(1, len(pop)):
        tObj = calculateObj(pop[i])
        if tObj < bestObj:
            bestObj = tObj
            bestInd = i

    return bestInd, bestObj

# **Main**

In [25]:
########### main #############

### Step 0 ###
# Number of population
Ps = 5

# Probability of crossover
Pc = 1.0

# Probability of mutation
Pm = 1.0

# Probability for tournament selection
Pto = 1.0

# Probability for roulette wheel selection
Pro = 1.0

# Probability for two-point crossover
Pt = 1.0

# Probability for PMX crossover
Pp = 1.0

# Probability for SJOX crossover
Psj = 1.0

# Probability for LOX crossover
Pl = 1.0

# Probability for Arbitrary three-job change mutation
P3j = 1.0

# Probability for Arbitrary two-job change mutation
P2j = 1.0

# Probability for Shift change mutation
Psh = 1.0

# Constant for mutation multiplier
E = 2

# Generation limit for inserting new randomly generated chromosomes
Gp = 50

# Generation limit constant for changing mutation probability
Gm = 30


k = 1


# Stopping number for generation
Ng = 100


# Creating the initial population
population = initialization(Ps)

population

[[8, 7, 2, 0, 3, 9, 1, 10, 4, 5, 6],
 [4, 0, 5, 3, 10, 8, 1, 2, 6, 7, 9],
 [7, 0, 6, 3, 8, 2, 5, 4, 9, 10, 1],
 [4, 5, 1, 8, 10, 6, 9, 3, 7, 0, 2],
 [6, 10, 2, 4, 1, 3, 8, 5, 0, 9, 7]]

In [26]:
### Step 1 ###
C_max = []
for i in population:
    C_max1 = calculateObj(i)
    C_max.append(C_max1)
pop_Cmax = list(zip(population,C_max))

sorted_pop_Cmax = sorted(pop_Cmax, key=lambda x: x[1])

X_s = sorted_pop_Cmax[0][0]
X_ss = sorted_pop_Cmax[1][0]

print("Xs = ", X_s)
print("Xss = ", X_ss)

Xs =  [4, 5, 1, 8, 10, 6, 9, 3, 7, 0, 2]
Xss =  [8, 7, 2, 0, 3, 9, 1, 10, 4, 5, 6]


In [27]:
### Step 2 ###
Mk = [sorted_pop_Cmax[0][1]]


population = [element[0] for element in sorted_pop_Cmax]

print("Sorted population: ", population)
print("Mk: ", Mk)

Sorted population:  [[4, 5, 1, 8, 10, 6, 9, 3, 7, 0, 2], [8, 7, 2, 0, 3, 9, 1, 10, 4, 5, 6], [4, 0, 5, 3, 10, 8, 1, 2, 6, 7, 9], [6, 10, 2, 4, 1, 3, 8, 5, 0, 9, 7], [7, 0, 6, 3, 8, 2, 5, 4, 9, 10, 1]]
Mk:  [197]


In [28]:
Mk

[197]

In [32]:
# Run the algorithm for 'stopGeneration' times generation
### Step 3 ###
countmut = 0
cntrndpop = 0

while k < Ng:
    childs = []

    Mk = [10000]
    if len(Mk) > 1:
        if Mk[0] == Mk[1]:
            countmut += 1
            cntrndpop += 1
        else:
            countmut = 0
            cntrndpop = 0
    else:
      countmut = 0
      cntrndpop = 0


    ### Step 4 ###

    q = 0
    while (q < (Ps//2)):


    ### Step 5 ###
        # Selecting parents
        r = random.random()
        if r <= Pto:
            parents = tournament_selection(population, tournament_size = 2)
        else:
            parents = roulette_wheel_selection(population)



    ### Step 6 ###

        cross_temp = []
        # Apply crossover
        r = random.random()
        if r <= Pc:
            r = random.random()
            if r <= Pt:
                cross_temp.append(two_point_crossover(parents))
            elif Pt < r <= (Pt + Pp):
                cross_temp.append(partially_mapped_crossover(parents))
            elif (Pt + Pp) < r <= (Pt + Pp + Psj):
                cross_temp.append(similar_job_order_crossover(parents))
            else:
                cross_temp.append(linearly_order_crossover(parents))

        else:
            cross_temp.append(parents)



    ### Step 7 ###
        # Apply mutation

        r = random.random()
        if r <= Pm:
            r = random.random()
            if r <= P3j:
                c1 = three_job_change(cross_temp[0][0])
                c2 = three_job_change(cross_temp[0][1])
            elif P3j < r <= (P3j + P2j):
                c1 = two_job_change(cross_temp[0][0])
                c2 = two_job_change(cross_temp[0][1])
            else:
                c1 = shift_change(cross_temp[0][0])
                c2 = shift_change(cross_temp[0][1])
            cross_temp[0][0] = c1
            cross_temp[0][1] = c2
        else:
            cross_temp[0][0] = c1
            cross_temp[0][1] = c2

        q += 1



    ### Step 9 ###

    if countmut > Gm:
        Pm = E * Pm
    else:
        Pm = Pm



    ### Stpe 10 ###

    if cntrndpop > Gp:
        new_Ps = int(0.75 * Ps)
        new_population = initialization(new_Ps)
        population = new_population + population[new_Ps:]
        cntrndpop = 0



    ### Step 11 ###

    C_max = []

    for i in range(Ps):
        C_max1 = calculateObj(population[i])
        C_max.append(C_max1)

    pop_Cmax = list(zip(population,C_max))

    sorted_pop_Cmax = sorted(pop_Cmax, key=lambda x: x[1])

    X_s = sorted_pop_Cmax[0][0]
    X_ss = sorted_pop_Cmax[1][0]

    Mk = [sorted_pop_Cmax[0][1]]






    ### Step 12 ###
    # Update the population

#    population = elitistUpdate(population, childs)

    population.append(c1)
    population.append(c2)

    ### Step 13 ###
    k += 1


    ### Step 14 ###

    bestSol, bestObj = findBestSolution(population)

    Mk = Mk.append(bestObj)

In [33]:
print("Solution:")
print(population[bestSol])


print("Objective Value:")
print(bestObj)

Solution:
[8, 7, 1, 5, 2, 10, 9, 0, 4, 3, 6]
Objective Value:
178
